# Experimentos com dados

Setup: https://github.com/lucasew/uv-jupyter-kernel
Python: 3.10

In [17]:
!uv pip install --no-progress requests pandas

Resolved 11 packages in 351ms
Prepared 1 package in 574ms
Installed 4 packages in 97ms
 + numpy==2.2.6
 + pandas==2.3.3
 + pytz==2025.2
 + tzdata==2025.2


In [19]:
import requests
import pandas as pd
from collections import defaultdict

In [75]:
def get_weather_data(lat, lon, start, end):
    data = requests.get(
        'https://power.larc.nasa.gov/api/temporal/hourly/point',
         params={
                'community': 'ag',
                'parameters': 'T2M,RH2M,WS10M,T2MDEW,PRECTOTCORR',
                'latitude': lat,
                'longitude': lon,
                'start': start,
                'end': end,
                'format': 'json',
                'units': 'metric',
                'header': 'true'
         }
    ).json()
    # print(data)

    items = defaultdict(lambda: {})
    for k, v in data['properties']['parameter'].items():
        for dt, val in v.items():
            items[dt][k] = val
            items[dt]['date'] = dt
            items[dt]['latitude'] = lat
            items[dt]['longitude'] = lon

    def fill_value_to_nan(item):
        if item == data['header']['fill_value']:
            return float('nan')
        return item
    return pd.DataFrame(items.values()).map(fill_value_to_nan).dropna()


In [73]:
get_weather_data(-23, -52, '20250903', '20250904')

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-52.0, -23.0, 442.02]}, 'properties': {'parameter': {'T2M': {'2025090300': 22.17, '2025090301': 21.46, '2025090302': 20.81, '2025090303': 20.09, '2025090304': 19.37, '2025090305': 18.73, '2025090306': 18.26, '2025090307': 18.73, '2025090308': 21.2, '2025090309': 24.71, '2025090310': 28.16, '2025090311': 30.64, '2025090312': 32.41, '2025090313': 33.54, '2025090314': 34.1, '2025090315': 34.04, '2025090316': 33.35, '2025090317': 31.95, '2025090318': 29.35, '2025090319': 27.26, '2025090320': 26.1, '2025090321': 25.23, '2025090322': 24.5, '2025090323': 23.91, '2025090400': 23.31, '2025090401': 22.59, '2025090402': 21.9, '2025090403': 21.28, '2025090404': 20.75, '2025090405': 20.18, '2025090406': 19.58, '2025090407': 20.32, '2025090408': 22.59, '2025090409': 26.45, '2025090410': 29.88, '2025090411': 31.89, '2025090412': 33.23, '2025090413': 34.11, '2025090414': 34.52, '2025090415': 34.35, '2025090416': 33.62, '2025090417': 32.

,T2M,date,latitude,longitude,RH2M,WS10M,T2MDEW,PRECTOTCORR
0,22.17,2025090300,-23,-52,50.00,3.43,11.29,0.0
1,21.46,2025090301,-23,-52,51.96,3.64,11.23,0.0
2,20.81,2025090302,-23,-52,54.68,4.20,11.42,0.0
3,20.09,2025090303,-23,-52,59.77,4.85,12.12,0.0
4,19.37,2025090304,-23,-52,67.69,5.32,13.34,0.0
5,18.73,2025090305,-23,-52,76.56,5.52,14.58,0.0
6,18.26,2025090306,-23,-52,82.62,5.63,15.27,0.0
7,18.73,2025090307,-23,-52,81.20,6.11,15.47,0.0
8,21.20,2025090308,-23,-52,67.97,6.28,15.15,0.0
9,24.71,2025090309,-23,-52,51.69,6.49,14.17,0.0


In [74]:
get_weather_data(22, 22, '20251003', '20251003').dropna()

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [22.0, 22.0, 516.44]}, 'properties': {'parameter': {'T2M': {'2025100300': -999.0, '2025100301': -999.0, '2025100302': -999.0, '2025100303': -999.0, '2025100304': -999.0, '2025100305': -999.0, '2025100306': -999.0, '2025100307': -999.0, '2025100308': -999.0, '2025100309': -999.0, '2025100310': -999.0, '2025100311': -999.0, '2025100312': -999.0, '2025100313': -999.0, '2025100314': -999.0, '2025100315': -999.0, '2025100316': -999.0, '2025100317': -999.0, '2025100318': -999.0, '2025100319': -999.0, '2025100320': -999.0, '2025100321': -999.0, '2025100322': -999.0, '2025100323': -999.0}, 'RH2M': {'2025100300': -999.0, '2025100301': -999.0, '2025100302': -999.0, '2025100303': -999.0, '2025100304': -999.0, '2025100305': -999.0, '2025100306': -999.0, '2025100307': -999.0, '2025100308': -999.0, '2025100309': -999.0, '2025100310': -999.0, '2025100311': -999.0, '2025100312': -999.0, '2025100313': -999.0, '2025100314': -999.0, '202510

,T2M,date,latitude,longitude,RH2M,WS10M,T2MDEW,PRECTOTCORR
